In [1]:
import spacy
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import json
import spacy_udpipe
from langdetect import detect
import string

nlp_es = spacy.load('es_core_news_sm')
nlp_ca = spacy.load('ca_core_news_sm')

In [2]:
f=open("training.json")
training_set=json.load(f)

f2=open("test.json")
test_set=json.load(f2)

In [3]:
l_training=[]
for i in range(len(training_set)):
    l_training.append({"text":training_set[i]["data"]["text"], "results":training_set[i]["predictions"][0]["result"]})

l_test=[]
for i in range(len(test_set)):
    l_test.append({"text":test_set[i]["data"]["text"], "results":test_set[i]["predictions"][0]["result"]})

In [4]:
def eliminate_patient_information(string):
    index = string.find("informe")
    if index != -1:
        return string[index:], index
    else:
        return string, 0

def eliminate_final_information(string):
    index = string.find("destinacio a l'alta")
    if index != -1:
        return string[:index]
    else:
        return string

def nhc_lopd(string):
    nhc_indices = [i for i in range(len(string)) if string.startswith("nhc", i)]
    lopd_indices = [i+len("lopd") for i in range(len(string)) if string.startswith("lopd", i)]

    results = []
    for nhc_index in nhc_indices:
        for lopd_index in lopd_indices:
            if nhc_index < lopd_index:
                length = lopd_index - nhc_index 
                results.append((nhc_index, length))
                break  # break inner loop, move to next nhc_index

    return results

def correct_results(results, lenght, nhclopd):
    for i in range(len(results)):
        results[i]["value"]["start"]-=lenght
        results[i]["value"]["end"]-=lenght
        for j in nhclopd:
            if(j[0]<results[i]["value"]["start"]):
                results[i]["value"]["start"]-=j[1]
                results[i]["value"]["end"]-=j[1]
    return results

def preprocessing(text_dict):
    text=text_dict["text"]
    results=text_dict["results"]
    text, lenght=eliminate_patient_information(text)
    text=eliminate_final_information(text)
    nhclopd=nhc_lopd(text)
    for i in (nhclopd):
        index1=i[0]
        index2=i[0]+i[1]
        text=text[0:index1]+text[index2:]
    results=correct_results(results, lenght, nhclopd)
    text_dict["text"]=text
    text_dict["results"]=results
    return text_dict


In [5]:
l_training_pro=[]
for i in range(len(l_training)):
    l_training_pro.append(preprocessing(l_training[i]))

l_test_pro=[]
for i in range(len(l_test)):
    l_test_pro.append(preprocessing(l_test[i]))

In [6]:
l_neg=[]
l_unc=[]
l_nsco=[]
l_usco=[]
for dic in l_training_pro:
    for dic2 in dic["results"]:
        if(dic2["value"]["labels"][0]=="NEG"):
            l_neg.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])
        
        elif(dic2["value"]["labels"][0]=="UNC"):
            l_unc.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])
        
        elif(dic2["value"]["labels"][0]=="NSCO"):
            l_nsco.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])
        
        elif(dic2["value"]["labels"][0]=="USCO"):
            l_usco.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])

In [7]:
for i in range(len(l_neg)):
    if(not l_neg[i][0].isalpha()):
        l_neg[i]=l_neg[i][1:]
    if(not l_neg[i][-1].isalpha()):
        l_neg[i]=l_neg[i][:-1]

for i in range(len(l_unc)):
    if(not l_unc[i][0].isalpha()):
        l_unc[i]=l_unc[i][1:]
    if(not l_unc[i][-1].isalpha()):
        l_unc[i]=l_unc[i][:-1]

for i in range(len(l_nsco)):
    if(not l_nsco[i][0].isalpha()):
        l_nsco[i]=l_nsco[i][1:]
    if(not l_nsco[i][-1].isalpha()):
        l_nsco[i]=l_nsco[i][:-1]

for i in range(len(l_usco)):
    if(not l_usco[i][0].isalpha()):
        l_usco[i]=l_usco[i][1:]
    if(not l_usco[i][-1].isalpha()):
        l_usco[i]=l_usco[i][:-1]

In [8]:
l_neg=list(set(l_neg))
l_unc=list(set(l_unc))
l_nsco=list(set(l_nsco))
l_usco=list(set(l_usco))

In [9]:
def BIO_tagger(text, results):
    l_text=word_tokenize(text)
    pos=[0,]
    l_neg=[]
    l_unc=[]
    l_starts=[x["value"]["start"] for x in results]
    l_ends=[x["value"]["end"] for x in results]
    l_labels=[x["value"]["labels"][0] for x in results]
    for i in range(len(l_text)-1):
        pos.append(pos[i]+len(l_text[i]))
        if(text[pos[-2]+len(text[pos[-2]])]==" "):
            pos[-1]+=1    
    for i in range(len(text)):
        if(i in pos):
            if(i in l_starts):
                index=l_starts.index(i)
                label=l_labels[index]
                if(label=="NEG"):
                    l_neg.append("B")
                    l_unc.append("O")
                elif(label=="NSCO"):
                    l_neg.append("I")
                    l_unc.append("O")
                elif(label=="UNC"):
                    l_neg.append("O")
                    l_unc.append("B")
                elif(label=="USCO"):
                    l_neg.append("O")
                    l_unc.append("I")
            else:
                x=False
                for j in range(len(l_starts)):
                    if(l_starts[j]<i and i<l_ends[j]):
                        label=l_labels[j]
                        if(label=="NEG"):
                            l_neg.append("B")
                            l_unc.append("O")
                        elif(label=="NSCO"):
                            l_neg.append("I")
                            l_unc.append("O")
                        elif(label=="UNC"):
                            l_neg.append("O")
                            l_unc.append("B")
                        elif(label=="USCO"):
                            l_neg.append("O")
                            l_unc.append("I")
                        x=True
                        break
                if not x:
                    l_neg.append("O")
                    l_unc.append("O")
    return [(word, tag) for word, tag in zip(l_text, l_neg)], [(word, tag) for word, tag in zip(l_text, l_unc)]

In [10]:
def POS(sentence, word):
    """
    Function that returns the POS tag of a word in a given sentence in Spanish or Catalan.

    Args:
        sentence (str): The input sentence.
        word (str): The word for which the POS tag is needed.

    Returns:
    str: The POS tag of the word
    """
    doc = None
    language = detect(sentence)

    if language == "ca":
        doc = nlp_ca(sentence)
    
    else:
        doc = nlp_es(sentence)

    for token in doc:
        if token.text.lower() == word.lower():
            return token.pos_

def init_cap(word):
    """
    Function to check if a word starts with an uppercase letter.

    Args:
    - word (str): The word to be checked.

    Returns:
    - bool: True if the word starts with an uppercase letter, False otherwise.
    """
    if word[0].capitalize() == word[0]:
        return True
    else:
        return False
    
    
def alphanum(word):
    """
    Function to check if a word consists of alphanumeric characters.
    
    Args:
    - word (str): The word to be checked.
    
    Returns:
    - bool: True if the word consists of alphanumeric characters, False otherwise.
    """
    return word.isalnum()

def has_num(word):
    """
    Function to check if a word contains a number.
    
    Args:
    - word (str): The word to be checked.
    
    Returns:
    - bool: True if the word contains a number, False otherwise.
    """
    return any(char.isdigit() for char in word)

def has_cap(word):
    """
    Function to check if a word contains a capitalized letter.
    
    Args:
    - word (str): The word to be checked.
    
    Returns:
    - bool: True if the word contains a capitalized letter, False otherwise.
    """
    return any(char.isupper() for char in word)

def has_dash(word):
    """
    Function to check if a word contains a dash (-).
    
    Args:
    - word (str): The word to be checked.
    
    Returns:
    - bool: True if the word contains a dash, False otherwise.
    """
    return '-' in word

def has_us(word):
    """
    Function to check if a word contains an underscore (_).
    
    Args:
    - word (str): The word to be checked.
    
    Returns:
    - bool: True if the word contains an underscore, False otherwise.
    """
    return '_' in word

def punctuation(word):
    """
    Function to check if a word contains punctuation.
    
    Args:
    - word (str): The word to be checked.
    
    Returns:
    - bool: True if the word contains punctuation, False otherwise.
    """
    return any(char in string.punctuation for char in word)

def suf_n(word, l_sufix):
    for i in range(-6,0):
        for s in l_sufix:
            if(len(s)==(-i) and len(word)>=(-i) and s in word[i:]):
                return s
    return ""

def pre_n(word, l_prefix):
    for i in range(4,-1,-1):
        for p in l_prefix:
            if(len(p)==(i+1) and len(word)>=i+1 and p in word[0:i+1]):
                return p
    return ""

def bigram_before(sentence, word, max_words=6):
    """
    Function that returns the bigram of up to 6 words before the observed word in a given sentence.

    Args:
        sentence (str): The input sentence.
        word (str): The word up to which the bigrams are needed.
        max_words (int): The maximum number of words before the observed word to consider. Default is 6.

    Returns:
        list: All bigrams before the observed word.
    """
    
    doc = None
    language = detect(sentence)
    if language == "ca":
        doc = nlp_ca(sentence)
    else:
        doc = nlp_es(sentence)

    bigrams = []
    for token in doc:
        if token.text.lower() == word.lower():
            index = token.i
            for i in range(max(0, index - max_words), index):  #Select the maxium len possible.     
                if i < index - 1:  
                    bigram = [doc[i].text, doc[i + 1].text] 
                    bigrams.append(bigram) 
            break

    return bigrams

def bigram_after(sentence, word):
    """
    Function that returns the POS tag of the word after the observed word in a given sentence.

    Args:
        sentence (str): The input sentence.
        word (str): The word for which the POS tag of the word after it is needed.

    Returns:
        str: The POS tag of the word after the observed word.
    """
    doc = None
    language = detect(sentence)
    if language == "ca":
        doc = nlp_ca(sentence)
    else:
        doc = nlp_es(sentence)

    if str(doc[-1])==".": 
        doc = doc[:-1]

    for token in doc:
        if token.text.lower() == word.lower():
            index = token.i
            # Check if there is a word after the observed word
            if index + 1 < len(doc):
                return [doc[index].text, doc[index+1].text]
            else:
                return []

def before_pos(sentence, word, max_words=6):
    """
    Function that returns the POS tags of up to 6 words before the observed word in a given sentence.

    Args:
        sentence (str): The input sentence.
        word (str): The word for which the POS tags of words before it are needed.
        max_words (int): The maximum number of words before the observed word to consider. Default is 6.

    Returns:
        list: The POS tags of words before the observed word.
    """
    doc = None
    language = detect(sentence)

    if language == "ca":
        doc = nlp_ca(sentence)
    
    else:
        doc = nlp_es(sentence)

    pos_tags = []
    for token in doc:
        if token.text.lower() == word.lower():
            index = token.i
            for i in range(max(0, index - max_words), index):  #Select the maxium len possible.
                pos_tags.append(doc[i].pos_)
            break

    return pos_tags

def after_pos(sentence, word):
    """
    Function that returns the POS tag of the word after the observed word in a given sentence.

    Args:
        sentence (str): The input sentence.
        word (str): The word for which the POS tag of the word after it is needed.

    Returns:
        str: The POS tag of the word after the observed word.
    """
    doc = None
    language = detect(sentence)
    if language == "es":
        doc = nlp_es(sentence)
    elif language == "ca":
        doc = nlp_ca(sentence)

    for token in doc:
        if token.text.lower() == word.lower():
            index = token.i
            # Check if there is a word after the observed word
            if index + 1 < len(doc):
                return doc[index + 1].pos_
            else:
                return []

def check_special_words(text, target):
    """
    Verify if a word exist in the special word vocabulary or sequence.

    Args:
        text (str): text to verify.

    Returns:
        list: boolean list.
    """

    # Define el conjunto de palabras especiales y secuencias especiales
    special_words = {
        "nada", "ni", "nunca", "ningun", "ninguno", "ninguna", "alguna", "apenas",
    }
    special_sequences = {
        "para nada", "ni siquiera"
    }
 
    words = word_tokenize(text)

    for i in range(len(words)):
        if words[i] == target:
            target_idx = i 

    result = []

    next = True
    for i in range(len(words)):
        word = words[i]
        #print(" ".join(words[i:i+2]),i,len(words))
        if next == False:
            next= True
            pass

        elif i < len(words) - 1 and " ".join(words[i:i+2]) in special_sequences:
            #print("special sequence:",len(words) - 1 and " ".join(words[i:i+2]))
            result.append(True)
            result.append(True)
            next = False
            # Skip one word 
            # i += 1
        
        elif word in special_words:
            result.append(True)
        
        else:
            result.append(False)

    res_index = None  
    for i in range(len(result)):
        if words[i] == target and target_idx == i:
            res_index = i
            break
    return result[res_index]

In [11]:
def extractFeatures(text, results, l_sufix, l_prefix):
    features={}
    l_neg_tagged, l_unc_tagged=BIO_tagger(text,results)
    features["text_tagged_neg"]=l_neg_tagged
    features["text_tagged_unc"]=l_unc_tagged
    features["words"]=[]
    for sentence in sent_tokenize(text):
        if any(char.isalpha() for char in sentence):
            for word in word_tokenize(sentence):
                dict_word={}
                dict_word["POS"]=POS(sentence, word)
                dict_word["init_cap"]=init_cap(word)
                dict_word["alphanum"]=alphanum(word)
                dict_word["has_num"]=has_num(word)
                dict_word["has_cap"]=has_cap(word)
                dict_word["has_dash"]=has_dash(word)
                dict_word["has_us"]=has_us(word)
                dict_word["punctuation"]=punctuation(word)
                dict_word["suf_n"]=suf_n(word, l_sufix)
                dict_word["pre_n"]=pre_n(word, l_prefix)
                dict_word["bigram_before"]=bigram_before(sentence, word)
                dict_word["bigram_after"]=bigram_after(sentence, word)
                dict_word["before_pos"]=before_pos(sentence, word)
                dict_word["check_special_words"]=check_special_words(sentence, word)
                features["words"].append({word:dict_word})
    return features

In [12]:
l_prefix=["a","an", "anti", "contra", "des", "dis", "de", "ex", "extra", "in", "im", "i", "ante", "circum", 
            "circun", "entre", "inter", "infra", "pos", "post", "sub", "so", "super", "sobre", "trans", "tras", 
            "ultra", "vice", "ante", "pre", "archi", "extra", "hiper", "re", "super", "ultra", "bi", "bis", "mini", "mono", "mon", 
            "multi", "pluri", "semi", "uni", "a", "ab", "amfi", "ana", "ante", "apo", "avant", "arxi", "ben", "auto", "bes",
            "bi", "circum", "co", "cat", "cata", "contra", "de", "des", "di", "dia", "en",
            "endo", "eso", "ex", "exo", "extra", "eu", "hemi", "hiper", "hipo", "fora", "in",
            "infra", "inter", "intro", "macro", "mal", "mega", "meta", "micro", "menys", "mono", "multi",
            "neo", "no", "pan", "par", "para", "pen", "per", "peri", "pluri", "poli", "post", "pre" ,"pro",
            "prop", "proto", "pseudo", "quasi", "re", "retro", "semi", "sin", "sota", "sub", "super", 
            "supra", "sus", "trans", "ultra", "vice"]

l_sufix=["ón", "ote", "ota", "azo", "aza", "ato", "ata", "aca", "aco", "udo", 
         "uda", "arrón", "ar", "ear", "ificar", "izar", "ecer", "ancia", "encia", 
         "anza", "ción", "sión", "ismo", "dad", "tad", "ada", "ería", "aje", "ez", 
         "mento", "miento", "dura", "oso", "osa", "ble", "able", "ible", "enco", "enca", 
         "ante", "iente", "ente", "ivo", "iva", "ano", "ana", "ado", "ada", "ido", "ida", "il", 
         "esco", "esca", "iento", "ienta", "oide", "izo", "ento", "mente", "aca", "ada", "al", "all", "alla", "am", "ar", "ari", "at", "ea", "atge", "eda", "et",
         "er", "era", "à", "ana", "ista", "aire", "ia", "eria", "isme", "esa",
         "eria", "or", "òria", "um", "uria", "im", "ícia", "ària", "ista",
         "alla", "ció", "ment", "menta", "nça", "atge", "ó", "dor", "dora", "all",
         "et", "il", "ístic", "ut", "uda", "enc", "enca", "ís", "issa", "ós", "osa",
         "aire", "ble", "ívol", "ívola", "dís", "egar", "ejar", "ificar", "itar", "itzar", "ment"]

In [14]:
test_set=[]
i=0
for dic in l_test_pro:
    test_set.append(extractFeatures(dic["text"], dic["results"], l_sufix, l_prefix))
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [15]:
with open("full_test_set_CRF.json","w") as f:
  json.dump(test_set,f)